In [1]:

import tensorflow as tf
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import PIL
import tensorflow.keras as keras


!rm -rf ./keras
train_data = np.zeros((165, 64*64))
train_labels = np.zeros((165,1), dtype= np.uint8)

for people in range(1,16):
    for face in range(1,12):
        if(people >= 10):
            path = './Yale2/subject%d_%d.bmp' % (people, face)
        else:
            path = './Yale2/subject0%d_%d.bmp' % (people, face)
        oriImage = PIL.Image.open(path)
        oriImage = oriImage.resize((64,64))
        imageArray = np.array(oriImage)
        imageArray = imageArray.T
        imageVec = np.reshape(imageArray, imageArray.shape[0] * imageArray.shape[1])
        # print(imageArray.shape)
        # print(imageBigDataORL.shape)
#             print((people - 1) * 10 + face)
        train_data[(people - 1) * 11 + face - 1] = imageVec
        train_labels[(people - 1) * 11 + face - 1] = people
    

print(train_data.shape)
print(train_labels.shape)
plt.figure()
plt.imshow(train_data[0].reshape([64,64]))

train_labels = train_labels - 1

np.random.seed(100)
train_data = np.random.permutation(train_data[50:,:])
np.random.seed(100)
train_labels = np.random.permutation(train_labels[50:])
test_data = train_data[0:50,:]
test_labels = train_labels[0:50]
np.random.seed(200)
test_data = np.random.permutation(test_data)
np.random.seed(200)
test_labels = np.random.permutation(test_labels)

#将标签转为one_hot类型
def label_to_one_hot(labels_dense, num_classes=15):
    num_labels = labels_dense.shape[0]
    print(num_labels)
    index_offset = np.arange(num_labels) * num_classes
    print(index_offset)
    labels_one_hot = np.zeros((num_labels, num_classes))
    print(labels_dense.ravel())
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

#将图片转为灰度图
def to4d(img):
    return img.reshape(img.shape[0],64,64,1).astype(np.float32)/255

train_data = to4d(train_data)
train_labels = label_to_one_hot(train_labels,15)
test_data = to4d(test_data)
test_labels = label_to_one_hot(test_labels,15)
plt.figure()
plt.imshow(train_data[0,:,:,0])
plt.title(np.argmax(train_labels[0]))


(165, 4096)
(165, 1)
115
[   0   15   30   45   60   75   90  105  120  135  150  165  180  195
  210  225  240  255  270  285  300  315  330  345  360  375  390  405
  420  435  450  465  480  495  510  525  540  555  570  585  600  615
  630  645  660  675  690  705  720  735  750  765  780  795  810  825
  840  855  870  885  900  915  930  945  960  975  990 1005 1020 1035
 1050 1065 1080 1095 1110 1125 1140 1155 1170 1185 1200 1215 1230 1245
 1260 1275 1290 1305 1320 1335 1350 1365 1380 1395 1410 1425 1440 1455
 1470 1485 1500 1515 1530 1545 1560 1575 1590 1605 1620 1635 1650 1665
 1680 1695 1710]
[12 11  9  7 13 11  6  7  7  6 11  9  8  7 13  6 14 11  8 13  7  9 11  6
 10 11  5 13  5  7 14 13  7  6 11 13  6 10  5  4  5 14 12 11 11  8 10  8
 12  9 12  8 12 14  6 14 10 13 10  4  6  8  8  8  8 14 14  7  9 12 14 10
  6 12 11  9 13 10 10  8  9  5 14  9 12 12  5  4  7  4  7  4 12  5  6  9
 10  5 14  7  5 14 10  9 13  9 13  5  8 11 12 13 10  6  5]
50
[  0  15  30  45  60  75  90 105 120

Text(0.5,1,'12')

In [2]:




# conv1 = tf.layers.conv2d(inputs=xs,filters=32,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu)
# pool1 = tf.layers.max_pooling2d(conv1,pool_size=2,strides=2)
# conv2 = tf.layers.conv2d(pool1,filters=72,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu)
# pool2 = tf.layers.max_pooling2d(conv2,pool_size=2,strides=2)
# flat = tf.reshape(pool2,[-1,16*16*72])
# dense = tf.layers.dense(flat,512,tf.nn.relu)
# dropout = tf.nn.dropout(dense,keep_prob)
# output = tf.layers.dense(dropout,15)

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, 2, padding='same', activation='relu',name='conv1'))
model.add(keras.layers.MaxPooling2D(strides=2, name='pool1'))
model.add(keras.layers.Conv2D(72, 2, padding='same', activation='relu',name='conv2'))
model.add(keras.layers.MaxPooling2D(strides=2, name='pool2'))
model.add(keras.layers.Flatten(name='flatten'))
# model.add(keras.layers.Dense(400, activation='relu', name='hidden_layer_2'))
model.add(keras.layers.Dense(512, activation='relu',name='dense'))
model.add(keras.layers.Dropout(0.25, name='drop'))
model.add(keras.layers.Dense(15, activation='softmax',name='output'))
model.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(0.0001), metrics=['accuracy','mae'])



tb_cb = keras.callbacks.TensorBoard(log_dir='./keras',write_images = 1, histogram_freq=1,write_grads=1)

cbks = [tb_cb]
history = model.fit(train_data, train_labels, batch_size=115, nb_epoch=55, 
verbose=1, callbacks=cbks, validation_data=(test_data, test_labels))

model.summary()


Train on 115 samples, validate on 50 samples
Epoch 1/55
115/115 [==============================] - 1s 9ms/step - loss: 2.7118 - acc: 0.0435 - mean_absolute_error: 0.1245 - val_loss: 2.6272 - val_acc: 0.1800 - val_mean_absolute_error: 0.1237
Epoch 2/55
115/115 [==============================] - 1s 5ms/step - loss: 2.6233 - acc: 0.2174 - mean_absolute_error: 0.1236 - val_loss: 2.5667 - val_acc: 0.2000 - val_mean_absolute_error: 0.1231
Epoch 3/55
115/115 [==============================] - 1s 7ms/step - loss: 2.5571 - acc: 0.2870 - mean_absolute_error: 0.1230 - val_loss: 2.5149 - val_acc: 0.3400 - val_mean_absolute_error: 0.1225
Epoch 4/55
115/115 [==============================] - 1s 4ms/step - loss: 2.5094 - acc: 0.2783 - mean_absolute_error: 0.1224 - val_loss: 2.4682 - val_acc: 0.3800 - val_mean_absolute_error: 0.1220
Epoch 5/55
115/115 [==============================] - 1s 8ms/step - loss: 2.4655 - acc: 0.3130 - mean_absolute_error: 0.1219 - val_loss: 2.4241 - val_acc: 0.4200 - val_mea

115/115 [==============================] - 0s 4ms/step - loss: 1.1895 - acc: 0.8348 - mean_absolute_error: 0.0861 - val_loss: 1.0918 - val_acc: 0.9800 - val_mean_absolute_error: 0.0836
Epoch 43/55
115/115 [==============================] - 0s 4ms/step - loss: 1.1732 - acc: 0.8522 - mean_absolute_error: 0.0850 - val_loss: 1.0620 - val_acc: 0.9800 - val_mean_absolute_error: 0.0822
Epoch 44/55
115/115 [==============================] - 0s 4ms/step - loss: 1.1270 - acc: 0.8609 - mean_absolute_error: 0.0837 - val_loss: 1.0327 - val_acc: 0.9800 - val_mean_absolute_error: 0.0808
Epoch 45/55
115/115 [==============================] - 0s 4ms/step - loss: 1.0751 - acc: 0.8783 - mean_absolute_error: 0.0807 - val_loss: 1.0028 - val_acc: 0.9800 - val_mean_absolute_error: 0.0793
Epoch 46/55
115/115 [==============================] - 0s 4ms/step - loss: 1.0174 - acc: 0.8348 - mean_absolute_error: 0.0780 - val_loss: 0.9725 - val_acc: 0.9800 - val_mean_absolute_error: 0.0777
Epoch 47/55
115/115 [======

In [3]:
! tensorboard --logdir ./keras

TensorBoard 1.9.0 at http://KhazeusMBP.local:6006 (Press CTRL+C to quit)
^C
